In [1]:
import pandas as pd
import numpy as np
import math
import datetime

In [2]:
train_m = pd.read_csv('train_starting.csv', index_col=0)
test_m = pd.read_csv('test_starting.csv', index_col = 0)

In [3]:
print list(train_m.columns)

['id', 'date_account_created', 'timestamp_first_active', 'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser', 'country_destination', 'lag_account_created', 'lag_first_active', 'lag_account_created_first_active', 'bookings', 'population_in_thousands', 'sum_secs_elapsed', 'counts']


In [4]:
#the toremove is a list of columns to be removed from the original dataset
excl = list(train_m.columns)
toremove = ['id', 'date_account_created', 'timestamp_first_active', 'date_first_booking', 'population_in_thousands',
            'lag_account_created', 'lag_first_active', 'bookings', 'lag_account_created_first_active', 
            'country_destination'] 
map(lambda x: excl.remove(x), toremove)

[None, None, None, None, None, None, None, None, None, None]

In [5]:
train_xcl = train_m.loc[:, excl]
test_xcl = test_m.loc[:, excl]

In [6]:
print train_xcl.columns
print test_xcl.columns

Index([u'gender', u'age', u'signup_method', u'signup_flow', u'language',
       u'affiliate_channel', u'affiliate_provider', u'first_affiliate_tracked',
       u'signup_app', u'first_device_type', u'first_browser',
       u'sum_secs_elapsed', u'counts'],
      dtype='object')
Index([u'gender', u'age', u'signup_method', u'signup_flow', u'language',
       u'affiliate_channel', u'affiliate_provider', u'first_affiliate_tracked',
       u'signup_app', u'first_device_type', u'first_browser',
       u'sum_secs_elapsed', u'counts'],
      dtype='object')


In [7]:
train_y_lag_creat = train_m.loc[:,'lag_account_created']

In [8]:
train_y_lag_first_act = train_m.loc[:,'lag_first_active']

In [9]:
from sklearn.feature_extraction import DictVectorizer

frames = [train_xcl, test_xcl]
all_data = pd.concat(frames)
all_data = all_data.T.to_dict().values()
train_xdic = train_xcl.T.to_dict().values()
test_xdic = test_xcl.T.to_dict().values()

vec = DictVectorizer()
vec.fit(all_data)
train_xvec = vec.transform(train_xdic)
test_xvec = vec.transform(test_xdic)


/usr/local/lib/python2.7/site-packages/pandas/core/frame.py:840: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  "columns will be omitted.", UserWarning)


In [10]:
trainx_vecmiss = pd.DataFrame(train_xvec.toarray()).fillna(-1)
testx_vecmiss = pd.DataFrame(test_xvec.toarray()).fillna(-1)
print trainx_vecmiss.shape
print testx_vecmiss.shape

(213451, 151)
(62096, 151)


In [11]:
np.sqrt(151)

12.288205727444508

In [85]:
from sklearn import ensemble
import sklearn.grid_search as gs
randomForest = ensemble.RandomForestClassifier(n_jobs = -1, oob_score=True)

np.random.seed(1)
grid_para_forest = [{"n_estimators": [200], "max_features": [12, 'auto', 14]}]
grid_search_forest = gs.GridSearchCV(randomForest, grid_para_forest, scoring='accuracy', cv=5, n_jobs = -1)
grid_search_forest.fit(trainx_vecmiss, train_y_lag_creat)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid=[{'n_estimators': [200], 'max_features': [12, 'auto', 14]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [88]:
print grid_search_forest.best_estimator_
print grid_search_forest.best_score_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)
0.59701758249


In [89]:
#predict train
from sklearn import cross_validation

randomForeststack = ensemble.RandomForestClassifier(n_jobs = -1, oob_score=True, n_estimators = 200, \
                                                    max_features = 'auto' )

predtrain_lag_creat = cross_validation.cross_val_predict(randomForeststack, trainx_vecmiss, \
                                                         train_y_lag_creat, cv=5, n_jobs = -1)

In [90]:
predtrain_lag_creat.shape

(213451,)

In [91]:
train_y_lag_creat.shape

(213451,)

In [92]:
from sklearn import metrics
metrics.accuracy_score(train_y_lag_creat, predtrain_lag_creat)

0.59690514450623333

In [93]:
#predict test lag account created
predtest_lag_creat = grid_search_forest.predict(testx_vecmiss)

print predtest_lag_creat.shape
print predtest_lag_creat[0:5]

(62096,)
['NB' 'NB' 'NB' 'NB' 'NB']


In [94]:
from sklearn import ensemble
import sklearn.grid_search as gs
randomForest_lag_first_act = ensemble.RandomForestClassifier(n_jobs = -1, oob_score=True)

np.random.seed(1)
grid_para_forest_act = [{"n_estimators": [200], "max_features": ['auto']}]
grid_search_forest_first_act = gs.GridSearchCV(randomForest_lag_first_act, grid_para_forest_act, scoring='accuracy', \
                                     cv=5, n_jobs = -1)
grid_search_forest_first_act.fit(trainx_vecmiss, train_y_lag_first_act)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid=[{'n_estimators': [200], 'max_features': ['auto']}],
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [95]:
grid_search_forest_first_act.best_score_

0.56761973474005745

In [96]:
print grid_search_forest_first_act.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)


In [97]:
randomForeststack_first_act = ensemble.RandomForestClassifier(n_jobs = -1, oob_score=True, n_estimators = 200, \
                                                    max_features = 'auto' )

predtrain_lag_first_act = cross_validation.cross_val_predict(randomForeststack_first_act, trainx_vecmiss, \
                                                         train_y_lag_first_act, cv=5, n_jobs = -1)

In [98]:
print train_y_lag_first_act.shape
print predtrain_lag_first_act.shape

(213451,)
(213451,)


In [99]:
predtest_lag_first_act = grid_search_forest.predict(testx_vecmiss)

print predtest_lag_first_act.shape
print predtest_lag_first_act[0:5]

(62096,)
['NB' 'NB' 'NB' 'NB' 'NB']


In [100]:
print testx_vecmiss.shape

(62096, 151)


In [ ]:
#predtrain_lag_creat
#predtest_lag_creat
#predtrain_lag_first_act
#predtest_lag_first_act

In [103]:
trainlags = pd.DataFrame({'id': train_m.id, 'predtrain_lag_creat':predtrain_lag_creat, \
                          'predtrain_lag_first_act':predtrain_lag_first_act})
#trainlags.to_csv("trainlags.csv", index=False)

In [104]:
testlags = pd.DataFrame({'id': test_m.id, 'predtest_lag_creat':predtest_lag_creat, \
                          'predtest_lag_first_act':predtest_lag_first_act})

#testlags.to_csv("testlags.csv", index=False)

In [186]:
trainbookingsdf = pd.read_csv('bkgs.stacked.train.csv', index_col=0)
testbookingsdf = pd.read_csv('bkgs.stacked.test.csv', index_col=0)

In [187]:
print trainbookingsdf.columns
print testbookingsdf.columns

Index([u'lag_account_created', u'lag_first_active',
       u'lag_account_created_first_active', u'gender', u'age',
       u'signup_method', u'signup_flow', u'language', u'affiliate_channel',
       u'affiliate_provider', u'first_affiliate_tracked', u'signup_app',
       u'first_device_type', u'first_browser', u'sum_secs_elapsed', u'counts',
       u'bookings', u'country_destination'],
      dtype='object')
Index([u'lag_account_created_first_active', u'gender', u'age',
       u'signup_method', u'signup_flow', u'language', u'affiliate_channel',
       u'affiliate_provider', u'first_affiliate_tracked', u'signup_app',
       u'first_device_type', u'first_browser', u'sum_secs_elapsed', u'counts',
       u'bookings'],
      dtype='object')


In [188]:
print trainbookingsdf.shape
print testbookingsdf.shape

(213451, 18)
(62096, 15)


In [191]:
exclupdated = list(train_m.columns)
toremoveupdated = [ 'date_account_created', 'timestamp_first_active', 'date_first_booking', 'population_in_thousands',
            'lag_account_created', 'lag_first_active', 'bookings', 'lag_account_created_first_active'] 
map(lambda x: exclupdated.remove(x), toremoveupdated)

[None, None, None, None, None, None, None, None]

In [192]:
exclupdated

['id',
 'gender',
 'age',
 'signup_method',
 'signup_flow',
 'language',
 'affiliate_channel',
 'affiliate_provider',
 'first_affiliate_tracked',
 'signup_app',
 'first_device_type',
 'first_browser',
 'country_destination',
 'sum_secs_elapsed',
 'counts']

In [209]:
updatedtrain = train_m.copy()
updatedtrain = updatedtrain.loc[:, exclupdated]
updatedtrain['pred_lag_account_created'] = predtrain_lag_creat
updatedtrain['pred_lag_first_active'] = predtrain_lag_first_act
updatedtrain['pred_bookings'] = list(trainbookingsdf.bookings[:])

In [210]:
print updatedtrain.shape
print updatedtrain.columns

(213451, 18)
Index([u'id', u'gender', u'age', u'signup_method', u'signup_flow', u'language',
       u'affiliate_channel', u'affiliate_provider', u'first_affiliate_tracked',
       u'signup_app', u'first_device_type', u'first_browser',
       u'country_destination', u'sum_secs_elapsed', u'counts',
       u'pred_lag_account_created', u'pred_lag_first_active',
       u'pred_bookings'],
      dtype='object')


In [211]:
updatedtrain.head(3)

,id,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,sum_secs_elapsed,counts,pred_lag_account_created,pred_lag_first_active,pred_bookings
0,gxn3p5htnn,NaN,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,NaN,NaN,NB,NB,NB
1,820tgsjxq7,MALE,35-39,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,NaN,NaN,same day,before,early
2,4ft3gnwmtx,FEMALE,55-59,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,NaN,NaN,greater 1 day,greater 1 day,NB


In [212]:
print train_m.columns

Index([u'id', u'date_account_created', u'timestamp_first_active',
       u'date_first_booking', u'gender', u'age', u'signup_method',
       u'signup_flow', u'language', u'affiliate_channel',
       u'affiliate_provider', u'first_affiliate_tracked', u'signup_app',
       u'first_device_type', u'first_browser', u'country_destination',
       u'lag_account_created', u'lag_first_active',
       u'lag_account_created_first_active', u'bookings',
       u'population_in_thousands', u'sum_secs_elapsed', u'counts'],
      dtype='object')


In [213]:
list(test_m.columns)

['id',
 'date_account_created',
 'timestamp_first_active',
 'date_first_booking',
 'gender',
 'age',
 'signup_method',
 'signup_flow',
 'language',
 'affiliate_channel',
 'affiliate_provider',
 'first_affiliate_tracked',
 'signup_app',
 'first_device_type',
 'first_browser',
 'lag_account_created_first_active',
 'sum_secs_elapsed',
 'counts']

In [214]:
exclupdatedtest = list(test_m.columns)
toremoveupdatedtest = [ 'date_account_created', 'timestamp_first_active', 'date_first_booking',
                   'lag_account_created_first_active'] 
map(lambda x: exclupdatedtest.remove(x), toremoveupdatedtest)

[None, None, None, None]

In [215]:
exclupdatedtest

['id',
 'gender',
 'age',
 'signup_method',
 'signup_flow',
 'language',
 'affiliate_channel',
 'affiliate_provider',
 'first_affiliate_tracked',
 'signup_app',
 'first_device_type',
 'first_browser',
 'sum_secs_elapsed',
 'counts']

In [219]:
testbookingsunfixed = list(testbookingsdf.bookings[:])
testbooking = []
for i in testbookingsunfixed:
    if i == 1:
        testbooking.append('early')
    elif i ==2:
        testbooking.append('NB')
    elif i ==3:
        testbooking.append('waited')
    else:
        print "error!!!!"

In [223]:
updatedtest = test_m.copy()
updatedtest = updatedtest.loc[:, exclupdatedtest]
updatedtest['pred_lag_account_created'] = predtest_lag_creat
updatedtest['pred_lag_first_active'] = predtest_lag_first_act
updatedtest['pred_bookings'] = testbooking

In [224]:
print updatedtest.columns
updatedtest.shape

Index([u'id', u'gender', u'age', u'signup_method', u'signup_flow', u'language',
       u'affiliate_channel', u'affiliate_provider', u'first_affiliate_tracked',
       u'signup_app', u'first_device_type', u'first_browser',
       u'sum_secs_elapsed', u'counts', u'pred_lag_account_created',
       u'pred_lag_first_active', u'pred_bookings'],
      dtype='object')


(62096, 17)

In [225]:
updatedtest.head(3)

,id,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,sum_secs_elapsed,counts,pred_lag_account_created,pred_lag_first_active,pred_bookings
0,5uwns89zht,FEMALE,35-39,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,119187.0,8.0,NB,NB,NB
1,jtl0dijy2j,NaN,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,250119.0,19.0,NB,NB,NB
2,xx0ulgorjt,NaN,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,975575.0,58.0,NB,NB,NB


In [226]:
#updatedtrain.to_csv("training_for_final_models.csv", index= 0)
#updatedtest.to_csv("test_for_final_models.csv", index = 0)